In [ ]:
# Balanced Variational Auto Encoders
# encoder_layers =  [[10, 8], [12, 10], [14, 12], [16, 14], [18, 16], [20, 18] ,[22,20]]
# latent_layer = [8, 10, 12, 14, 16, 18]
# decoder_layers = [[8, 10], [10, 12], [12, 14], [14, 16], [16, 18], [18, 20], [20,22]]
# EPOCHS = 150

In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use('ggplot')

In [2]:
src_df = pd.read_csv("Original.csv") # ../SB-001/SB-001
x_train = src_df.values
train_ds = torch.tensor(x_train).to(torch.float32)
print(train_ds.shape)
input_vector = src_df.shape[1]

torch.Size([57, 24])


In [3]:
input_vector

24

In [4]:
def plot_correlation_map(original_df, synth_df, name):

    corr_matrix1 = original_df.corr()
    corr_matrix2 = synth_df.corr()

    fig, axes = plt.subplots(1, 2, figsize=(18, 10))
    sns.heatmap(corr_matrix1, annot=True, cmap='coolwarm', ax=axes[0])
    axes[0].set_title('Heatmap 1')

    sns.heatmap(corr_matrix2, annot=True, cmap='coolwarm', ax=axes[1])
    axes[1].set_title('Heatmap 2')

    plt.tight_layout()
    plt.savefig(f"{name}_corelation.png")
    plt.clf()
    plt.close(fig)

def plot_density_curves(original_df, synth_df, name):
    for column in original_df.columns:
        fig, ax = plt.subplots()

        sns.kdeplot(data=original_df[column], color='blue', label='Original', ax=ax)
        sns.kdeplot(data=synth_df[column], color='green', label='Synthetic', ax=ax)

        ax.legend()
        plt.tight_layout()
        plt.savefig(f"{name}_{column}_density.png")
        plt.clf()
        plt.close(fig)

def mean_and_stddiv(original_df, synth_df, name):
    common_columns = set(original_df.columns) & set(synth_df.columns)
    results = {}

    for column in common_columns:
        mean_df1 = original_df[column].mean()
        std_df1 = original_df[column].std()
        mean_df2 = synth_df[column].mean()
        std_df2 = synth_df[column].std()

        results[column] = {'Mean_df1': mean_df1, 'Std_df1': std_df1,
                           'Mean_df2': mean_df2, 'Std_df2': std_df2}

    comparison_df = pd.DataFrame(results)
    comparison_df.to_csv(name + 'mean_std.csv', index=True)       
  

In [5]:
class autoencoder(nn.Module):
    def __init__(self, input_vector, encoder_layers, latent_layer, decoder_layers):
        super().__init__()
        
        """
        # Hyper params
        self.learning_rate = 0.01
        self.optimizer = optim.Adam(params=self.parameters(), lr=self.learning_rate)'
        """
        # Construct the layers of the encoder
        self.encoder = nn.ModuleList()
        self.encoder.append(nn.Linear(input_vector, encoder_layers[0]))
        for i in range(len(encoder_layers) - 1):
            self.encoder.append(nn.Linear(encoder_layers[i], encoder_layers[i+1]))
            self.encoder.append(nn.ReLU())
        #self.encoder.append(nn.Linear(encoder_layers[-1], latent_layer))
        #self.encoder.append(nn.BatchNorm(latent_layer)) # Will play with BatchNorm after getting this to work
        
        # Layers to make the latent vector
        self.encode_mean = nn.Linear(encoder_layers[-1], latent_layer)
        self.encode_variance = nn.Linear(encoder_layers[-1], latent_layer)
        
        # Construct the layers of the decoder
        self.decoder = nn.ModuleList()
        self.decoder.append(nn.Linear(latent_layer, decoder_layers[0]))
        for i in range(len(decoder_layers) - 1):
            self.decoder.append(nn.Linear(decoder_layers[i], decoder_layers[i+1]))
            self.decoder.append(nn.ReLU())
        self.decoder.append(nn.Linear(decoder_layers[-1], input_vector))

            # Method to run the encoder layers
    def encode(self, input_data):
        for layer in self.encoder:
            input_data = layer(input_data)
        mean = self.encode_mean(input_data)
        variance = self.encode_variance(input_data)
        return mean, variance
    
    # Resamples the distribution
    def reparameterize(self, mean, variance):
        #std = torch.exp(0.5 * variance)
        resampled_vector = mean + variance * torch.randn_like(variance)
        return resampled_vector
    
    # Method to run the decoder layers
    def decode(self, input_data):
        for layer in self.decoder:
            input_data = layer(input_data)
        return input_data
    
    # Run the autoencoder
    def forward(self, input_data):
        mean, variance = self.encode(input_data)
        resampled_vector = self.reparameterize(mean, variance)
        output = self.decode(resampled_vector)
        return output, mean, variance

def calc_loss(output_data, target, mean, variance):
    reconstruction_loss = nn.MSELoss(reduction='sum')(output_data, target)
    kl_div = -0.5 * torch.mean(torch.sum(1 + variance - mean ** 2 - torch.exp(variance), dim = 1), dim = 0)
    return reconstruction_loss + kl_div

In [6]:
encoder_layers =  [[10, 8], [12, 10], [14, 12], [16, 14], [18, 16], [20, 18] ,[22,20]]
latent_layer = [8, 10, 12, 14, 16, 18]
decoder_layers = [[8, 10], [10, 12], [12, 14], [14, 16], [16, 18], [18, 20], [20,22]]
EPOCHS = 150

In [8]:
for latent in latent_layer:
    for enc_layers in encoder_layers:
        for dec_layers in decoder_layers:
            print(f"L{input_vector}_E{enc_layers}_B{latent}_D{dec_layers}")

L24_E[10, 8]_B8_D[8, 10]
L24_E[10, 8]_B8_D[10, 12]
L24_E[10, 8]_B8_D[12, 14]
L24_E[10, 8]_B8_D[14, 16]
L24_E[10, 8]_B8_D[16, 18]
L24_E[10, 8]_B8_D[18, 20]
L24_E[10, 8]_B8_D[20, 22]
L24_E[12, 10]_B8_D[8, 10]
L24_E[12, 10]_B8_D[10, 12]
L24_E[12, 10]_B8_D[12, 14]
L24_E[12, 10]_B8_D[14, 16]
L24_E[12, 10]_B8_D[16, 18]
L24_E[12, 10]_B8_D[18, 20]
L24_E[12, 10]_B8_D[20, 22]
L24_E[14, 12]_B8_D[8, 10]
L24_E[14, 12]_B8_D[10, 12]
L24_E[14, 12]_B8_D[12, 14]
L24_E[14, 12]_B8_D[14, 16]
L24_E[14, 12]_B8_D[16, 18]
L24_E[14, 12]_B8_D[18, 20]
L24_E[14, 12]_B8_D[20, 22]
L24_E[16, 14]_B8_D[8, 10]
L24_E[16, 14]_B8_D[10, 12]
L24_E[16, 14]_B8_D[12, 14]
L24_E[16, 14]_B8_D[14, 16]
L24_E[16, 14]_B8_D[16, 18]
L24_E[16, 14]_B8_D[18, 20]
L24_E[16, 14]_B8_D[20, 22]
L24_E[18, 16]_B8_D[8, 10]
L24_E[18, 16]_B8_D[10, 12]
L24_E[18, 16]_B8_D[12, 14]
L24_E[18, 16]_B8_D[14, 16]
L24_E[18, 16]_B8_D[16, 18]
L24_E[18, 16]_B8_D[18, 20]
L24_E[18, 16]_B8_D[20, 22]
L24_E[20, 18]_B8_D[8, 10]
L24_E[20, 18]_B8_D[10, 12]
L24_E[20, 18]_

In [7]:
total_iterations = len(latent_layer) * len(encoder_layers) * len(decoder_layers)
print("Total Model in Pipeline:",total_iterations)

#print total models
import time
time.sleep(3)

for latent in latent_layer:
    for enc_layers in encoder_layers:
        for dec_layers in decoder_layers:
            print(f"TRAINING: L{input_vector}_E{enc_layers}_B{latent}_D{dec_layers}")
            model = autoencoder(input_vector, enc_layers, latent, dec_layers)

            optimizer = optim.Adam(model.parameters(), lr=0.001)
            train_dl = DataLoader(train_ds)

            history = []
            for epoch in range(EPOCHS):
                total_loss = 0
                for sample in train_dl:
                    # forward pass
                    pred, mean, variance = model(sample)
                    loss = calc_loss(pred, sample, mean, variance)
                    # backward pass
                    optimizer.zero_grad()
                    loss.backward()
                    optimizer.step()
                    
                    total_loss += loss.item()
                history.append([epoch, total_loss])    
                #print('Epoch:', epoch, '\t Loss:', total_loss)
            print("TRAINING COMPLETED: LOSS", history[-1][1])
            history = pd.DataFrame(history, columns=['epoch', 'loss'])
            name = f"L{input_vector}_E{enc_layers}_B{latent}_D{dec_layers}" 

            
            plt.plot(history['epoch'], history['loss'])
            plt.savefig(f"{name}_loss.png")
            plt.clf()

            original_df = pd.DataFrame(train_ds.numpy(), columns=src_df.columns)
            synth_data = model(train_ds)[0].detach().numpy()
            synth_df = pd.DataFrame(synth_data, columns=src_df.columns)
            
            history.to_csv(f"{name}_history.csv")
            synth_df.to_csv(f"{name}_synthetic.csv")

            print("DRAWING FIGURES")
            plot_correlation_map(original_df, synth_df, name)
            plot_density_curves(original_df, synth_df, name)
            mean_and_stddiv(original_df, synth_df, name)
            print("FIGURES DRAWN")
            plt.close()       

Total Model in Pipeline: 294
TRAINING: L24_E[10, 8]_B8_D[8, 10]
TRAINING COMPLETED: LOSS 19.89044465869665
DRAWING FIGURES
FIGURES DRAWN
TRAINING: L24_E[10, 8]_B8_D[10, 12]
TRAINING COMPLETED: LOSS 13.962103959172964
DRAWING FIGURES
FIGURES DRAWN
TRAINING: L24_E[10, 8]_B8_D[12, 14]
TRAINING COMPLETED: LOSS 12.414771564304829
DRAWING FIGURES
FIGURES DRAWN
TRAINING: L24_E[10, 8]_B8_D[14, 16]
TRAINING COMPLETED: LOSS 13.344386756420135
DRAWING FIGURES
FIGURES DRAWN
TRAINING: L24_E[10, 8]_B8_D[16, 18]
TRAINING COMPLETED: LOSS 17.439788408577442
DRAWING FIGURES
FIGURES DRAWN
TRAINING: L24_E[10, 8]_B8_D[18, 20]
TRAINING COMPLETED: LOSS 11.820133585482836
DRAWING FIGURES
FIGURES DRAWN
TRAINING: L24_E[10, 8]_B8_D[20, 22]
TRAINING COMPLETED: LOSS 14.917390815913677
DRAWING FIGURES
FIGURES DRAWN
TRAINING: L24_E[12, 10]_B8_D[8, 10]
TRAINING COMPLETED: LOSS 14.699948489665985
DRAWING FIGURES
FIGURES DRAWN
TRAINING: L24_E[12, 10]_B8_D[10, 12]
TRAINING COMPLETED: LOSS 14.016606122255325
DRAWING FIGU

In [ ]:
'''
Model	Loss
 L24_E[18, 16]_B8_D[20, 22]	7.173943959
 L24_E[22, 20]_B12_D[20, 22]	7.319492649
 L24_E[22, 20]_B12_D[14, 16]	7.553969376
 L24_E[18, 16]_B14_D[20, 22]	7.557667173
 L24_E[22, 20]_B14_D[14, 16]	8.335510902
 L24_E[22, 20]_B18_D[20, 22]	8.559720572
 L24_E[20, 18]_B14_D[14, 16]	8.650961135
 L24_E[22, 20]_B14_D[16, 18]	8.732573692
 L24_E[20, 18]_B10_D[20, 22]	8.80281369
 L24_E[22, 20]_B12_D[18, 20]	8.818260495
 L24_E[20, 18]_B16_D[18, 20]	9.050367374
 L24_E[20, 18]_B12_D[14, 16]	9.10349191
 L24_E[18, 16]_B18_D[20, 22]	9.109338995
 L24_E[22, 20]_B10_D[16, 18]	9.113454726
 L24_E[22, 20]_B8_D[20, 22]	9.118232332
 L24_E[18, 16]_B10_D[16, 18]	9.11971857
 L24_E[18, 16]_B14_D[12, 14]	9.221962765
 L24_E[16, 14]_B16_D[16, 18]	9.28784271
 L24_E[16, 14]_B10_D[20, 22]	9.292657308
 L24_E[18, 16]_B14_D[18, 20]	9.319958288
 L24_E[22, 20]_B10_D[20, 22]	9.32412133
 L24_E[18, 16]_B12_D[18, 20]	9.427634474
 L24_E[22, 20]_B10_D[18, 20]	9.434077147
 L24_E[22, 20]_B8_D[16, 18]	9.438784473
 L24_E[16, 14]_B16_D[12, 14]	9.546397701
 L24_E[22, 20]_B16_D[18, 20]	9.586508591
 L24_E[20, 18]_B8_D[14, 16]	9.615520891
 L24_E[20, 18]_B16_D[20, 22]	9.708080962
 L24_E[18, 16]_B10_D[18, 20]	9.728945594
 L24_E[20, 18]_B16_D[16, 18]	9.740140025
 L24_E[22, 20]_B14_D[20, 22]	9.768046677
 L24_E[16, 14]_B14_D[18, 20]	9.789381362
 L24_E[18, 16]_B18_D[16, 18]	9.804269146
 L24_E[16, 14]_B8_D[14, 16]	9.806049783
 L24_E[18, 16]_B8_D[14, 16]	9.812960811
 L24_E[20, 18]_B12_D[20, 22]	9.833591886
 L24_E[20, 18]_B14_D[18, 20]	9.884473901
 L24_E[16, 14]_B18_D[20, 22]	9.890682973
 L24_E[14, 12]_B18_D[16, 18]	9.892482951
 L24_E[18, 16]_B10_D[12, 14]	9.942912489
 L24_E[22, 20]_B16_D[14, 16]	10.03164958
 L24_E[14, 12]_B12_D[20, 22]	10.03606597
 L24_E[16, 14]_B10_D[18, 20]	10.06703123
 L24_E[18, 16]_B16_D[20, 22]	10.0785776
 L24_E[20, 18]_B14_D[20, 22]	10.08519817
 L24_E[22, 20]_B16_D[12, 14]	10.09101779
 L24_E[20, 18]_B18_D[18, 20]	10.16116649
 L24_E[20, 18]_B10_D[16, 18]	10.2221844
 L24_E[22, 20]_B18_D[12, 14]	10.28193089
 L24_E[20, 18]_B12_D[16, 18]	10.28975555
 L24_E[16, 14]_B18_D[10, 12]	10.29912125
 L24_E[22, 20]_B8_D[12, 14]	10.42594568
 L24_E[22, 20]_B14_D[18, 20]	10.44351257
 L24_E[14, 12]_B12_D[12, 14]	10.45644093
 L24_E[14, 12]_B14_D[16, 18]	10.4610449
 L24_E[12, 10]_B10_D[20, 22]	10.50409936
 L24_E[22, 20]_B8_D[18, 20]	10.56166777
 L24_E[22, 20]_B10_D[14, 16]	10.58195911
 L24_E[22, 20]_B16_D[20, 22]	10.59977185
 L24_E[22, 20]_B8_D[8, 10]	10.66086794
 L24_E[18, 16]_B12_D[16, 18]	10.68763087
 L24_E[20, 18]_B12_D[12, 14]	10.69370994
 L24_E[14, 12]_B12_D[14, 16]	10.71389473
 L24_E[12, 10]_B14_D[20, 22]	10.71794942
 L24_E[22, 20]_B18_D[16, 18]	10.75760443
 L24_E[14, 12]_B14_D[18, 20]	10.77038356
 L24_E[16, 14]_B14_D[16, 18]	10.77250651
 L24_E[20, 18]_B16_D[12, 14]	10.78925025
 L24_E[14, 12]_B14_D[8, 10]	10.80792279
 L24_E[12, 10]_B18_D[20, 22]	10.81136944
 L24_E[18, 16]_B18_D[18, 20]	10.84815505
 L24_E[16, 14]_B14_D[10, 12]	10.87388009
 L24_E[20, 18]_B10_D[14, 16]	10.87788909
 L24_E[10, 8]_B14_D[14, 16]	10.89672007
 L24_E[18, 16]_B16_D[18, 20]	10.89914908
 L24_E[22, 20]_B18_D[18, 20]	10.90930091
 L24_E[20, 18]_B18_D[20, 22]	10.93683643
 L24_E[14, 12]_B10_D[16, 18]	11.00262084
 L24_E[20, 18]_B14_D[10, 12]	11.03367847
 L24_E[10, 8]_B14_D[18, 20]	11.09267681
 L24_E[14, 12]_B10_D[20, 22]	11.12520647
 L24_E[20, 18]_B8_D[16, 18]	11.1519158
 L24_E[18, 16]_B12_D[20, 22]	11.16214188
 L24_E[18, 16]_B18_D[14, 16]	11.19376866
 L24_E[22, 20]_B12_D[16, 18]	11.23676279
 L24_E[20, 18]_B16_D[10, 12]	11.24141002
 L24_E[14, 12]_B18_D[14, 16]	11.27589469
 L24_E[18, 16]_B18_D[10, 12]	11.28569888
 L24_E[16, 14]_B12_D[12, 14]	11.30984944
 L24_E[16, 14]_B16_D[20, 22]	11.32175976
 L24_E[16, 14]_B14_D[20, 22]	11.33877602
 L24_E[18, 16]_B16_D[8, 10]	11.35928079
 L24_E[10, 8]_B16_D[20, 22]	11.38278206
 L24_E[20, 18]_B18_D[12, 14]	11.42157621
 L24_E[18, 16]_B8_D[18, 20]	11.43242968
 L24_E[22, 20]_B16_D[10, 12]	11.44544369
 L24_E[14, 12]_B8_D[12, 14]	11.45675384
 L24_E[16, 14]_B8_D[12, 14]	11.49658633
 L24_E[18, 16]_B12_D[10, 12]	11.55531573
 L24_E[14, 12]_B18_D[10, 12]	11.55995766
 L24_E[16, 14]_B18_D[8, 10]	11.57968571
 L24_E[18, 16]_B14_D[16, 18]	11.5875267
 L24_E[16, 14]_B16_D[14, 16]	11.60200046
 L24_E[16, 14]_B18_D[16, 18]	11.63877489
 L24_E[20, 18]_B10_D[18, 20]	11.66006492
 L24_E[16, 14]_B14_D[12, 14]	11.68785357
 L24_E[12, 10]_B18_D[18, 20]	11.76307507
 L24_E[16, 14]_B12_D[18, 20]	11.76606109
 L24_E[18, 16]_B10_D[20, 22]	11.78564965
 L24_E[16, 14]_B10_D[8, 10]	11.79712538
 L24_E[10, 8]_B8_D[18, 20]	11.82013359
 L24_E[18, 16]_B10_D[10, 12]	11.82695113
 L24_E[16, 14]_B16_D[18, 20]	11.82900938
 L24_E[22, 20]_B18_D[10, 12]	11.84687915
 L24_E[16, 14]_B18_D[12, 14]	11.85817574
 L24_E[20, 18]_B14_D[12, 14]	11.86014471
 L24_E[10, 8]_B16_D[18, 20]	11.86045638
 L24_E[16, 14]_B18_D[18, 20]	11.86323354
 L24_E[20, 18]_B12_D[10, 12]	11.87424756
 L24_E[12, 10]_B8_D[16, 18]	11.87641994
 L24_E[12, 10]_B14_D[18, 20]	11.88624621
 L24_E[10, 8]_B16_D[12, 14]	11.88902239
 L24_E[20, 18]_B18_D[16, 18]	11.89616188
 L24_E[22, 20]_B18_D[14, 16]	11.89972265
 L24_E[20, 18]_B18_D[14, 16]	11.93644732
 L24_E[16, 14]_B10_D[12, 14]	11.9731911
 L24_E[16, 14]_B18_D[14, 16]	11.98898218
 L24_E[10, 8]_B14_D[20, 22]	11.98960467
 L24_E[16, 14]_B14_D[14, 16]	11.99595478
 L24_E[18, 16]_B16_D[14, 16]	12.01023939
 L24_E[12, 10]_B12_D[18, 20]	12.02065571
 L24_E[12, 10]_B16_D[14, 16]	12.02592239
 L24_E[12, 10]_B18_D[14, 16]	12.05609915
 L24_E[22, 20]_B8_D[14, 16]	12.05762234
 L24_E[22, 20]_B12_D[12, 14]	12.07115354
 L24_E[18, 16]_B12_D[14, 16]	12.10494854
 L24_E[18, 16]_B16_D[10, 12]	12.11065836
 L24_E[18, 16]_B14_D[14, 16]	12.12387151
 L24_E[18, 16]_B8_D[8, 10]	12.1438164
 L24_E[12, 10]_B10_D[18, 20]	12.15990153
 L24_E[18, 16]_B18_D[12, 14]	12.16225525
 L24_E[16, 14]_B8_D[20, 22]	12.17325355
 L24_E[12, 10]_B14_D[12, 14]	12.18891843
 L24_E[14, 12]_B14_D[20, 22]	12.20754475
 L24_E[22, 20]_B12_D[10, 12]	12.25459632
 L24_E[22, 20]_B10_D[8, 10]	12.2638367
 L24_E[22, 20]_B16_D[16, 18]	12.31987515
 L24_E[20, 18]_B10_D[12, 14]	12.33749477
 L24_E[16, 14]_B16_D[8, 10]	12.37451407
 L24_E[14, 12]_B16_D[18, 20]	12.37540128
 L24_E[10, 8]_B8_D[12, 14]	12.41477156
 L24_E[12, 10]_B16_D[20, 22]	12.4401543
 L24_E[10, 8]_B18_D[20, 22]	12.44978818
 L24_E[10, 8]_B14_D[16, 18]	12.4507947
 L24_E[10, 8]_B18_D[8, 10]	12.47269165
 L24_E[14, 12]_B8_D[20, 22]	12.50067701
 L24_E[20, 18]_B8_D[18, 20]	12.50369526
 L24_E[16, 14]_B12_D[14, 16]	12.51954191
 L24_E[12, 10]_B10_D[12, 14]	12.52533775
 L24_E[18, 16]_B10_D[14, 16]	12.54679078
 L24_E[14, 12]_B16_D[16, 18]	12.55031394
 L24_E[10, 8]_B12_D[18, 20]	12.55151285
 L24_E[20, 18]_B14_D[16, 18]	12.5713681
 L24_E[10, 8]_B14_D[12, 14]	12.57231096
 L24_E[18, 16]_B14_D[10, 12]	12.59618385
 L24_E[16, 14]_B12_D[10, 12]	12.64062923
 L24_E[22, 20]_B14_D[12, 14]	12.6430424
 L24_E[16, 14]_B12_D[8, 10]	12.67445449
 L24_E[20, 18]_B16_D[8, 10]	12.7214115
 L24_E[22, 20]_B14_D[10, 12]	12.73417808
 L24_E[14, 12]_B10_D[8, 10]	12.73949144
 L24_E[20, 18]_B10_D[10, 12]	12.74313487
 L24_E[14, 12]_B18_D[12, 14]	12.75927999
 L24_E[16, 14]_B8_D[10, 12]	12.78757991
 L24_E[14, 12]_B8_D[8, 10]	12.7917805
 L24_E[20, 18]_B18_D[8, 10]	12.79363181
 L24_E[14, 12]_B18_D[20, 22]	12.81519832
 L24_E[12, 10]_B10_D[16, 18]	12.83910225
 L24_E[16, 14]_B10_D[10, 12]	12.84291209
 L24_E[14, 12]_B16_D[20, 22]	12.84547817
 L24_E[22, 20]_B8_D[10, 12]	12.8745778
 L24_E[22, 20]_B18_D[8, 10]	12.90914143
 L24_E[14, 12]_B16_D[8, 10]	12.91651969
 L24_E[12, 10]_B18_D[8, 10]	12.95243829
 L24_E[20, 18]_B8_D[10, 12]	12.95406342
 L24_E[12, 10]_B8_D[18, 20]	12.98539335
 L24_E[12, 10]_B10_D[14, 16]	12.9945853
 L24_E[14, 12]_B16_D[12, 14]	13.0038166
 L24_E[12, 10]_B18_D[10, 12]	13.01349178
 L24_E[12, 10]_B12_D[10, 12]	13.04377389
 L24_E[20, 18]_B16_D[14, 16]	13.04886166
 L24_E[22, 20]_B10_D[10, 12]	13.05666463
 L24_E[20, 18]_B8_D[20, 22]	13.07972345
 L24_E[16, 14]_B10_D[14, 16]	13.08909075
 L24_E[10, 8]_B10_D[12, 14]	13.12746833
 L24_E[12, 10]_B16_D[10, 12]	13.12882432
 L24_E[14, 12]_B14_D[14, 16]	13.15377965
 L24_E[14, 12]_B16_D[10, 12]	13.15611522
 L24_E[14, 12]_B8_D[18, 20]	13.19664016
 L24_E[20, 18]_B14_D[8, 10]	13.20693086
 L24_E[12, 10]_B12_D[8, 10]	13.21084749
 L24_E[18, 16]_B12_D[12, 14]	13.22677086
 L24_E[14, 12]_B12_D[18, 20]	13.23592228
 L24_E[14, 12]_B10_D[12, 14]	13.24981166
 L24_E[10, 8]_B12_D[12, 14]	13.27147979
 L24_E[22, 20]_B10_D[12, 14]	13.29601193
 L24_E[14, 12]_B10_D[10, 12]	13.32100116
 L24_E[10, 8]_B10_D[10, 12]	13.34303294
 L24_E[10, 8]_B8_D[14, 16]	13.34438676
 L24_E[10, 8]_B12_D[14, 16]	13.35319159
 L24_E[20, 18]_B18_D[10, 12]	13.36139749
 L24_E[16, 14]_B10_D[16, 18]	13.42777887
 L24_E[22, 20]_B14_D[8, 10]	13.44721503
 L24_E[14, 12]_B12_D[16, 18]	13.44794223
 L24_E[12, 10]_B8_D[20, 22]	13.46853535
 L24_E[14, 12]_B8_D[16, 18]	13.50640198
 L24_E[14, 12]_B14_D[10, 12]	13.50765466
 L24_E[18, 16]_B14_D[8, 10]	13.51755511
 L24_E[16, 14]_B16_D[10, 12]	13.53465617
 L24_E[12, 10]_B14_D[10, 12]	13.55504987
 L24_E[12, 10]_B12_D[12, 14]	13.55514862
 L24_E[18, 16]_B16_D[16, 18]	13.55869327
 L24_E[12, 10]_B12_D[16, 18]	13.57639434
 L24_E[10, 8]_B18_D[14, 16]	13.60415046
 L24_E[10, 8]_B16_D[16, 18]	13.60673361
 L24_E[10, 8]_B12_D[16, 18]	13.61289556
 L24_E[18, 16]_B16_D[12, 14]	13.62986945
 L24_E[20, 18]_B10_D[8, 10]	13.63902129
 L24_E[14, 12]_B18_D[8, 10]	13.65219187
 L24_E[16, 14]_B12_D[20, 22]	13.6625939
 L24_E[20, 18]_B12_D[18, 20]	13.68198017
 L24_E[16, 14]_B8_D[16, 18]	13.74270364
 L24_E[10, 8]_B18_D[18, 20]	13.7501398
 L24_E[14, 12]_B18_D[18, 20]	13.78162538
 L24_E[10, 8]_B16_D[10, 12]	13.8131934
 L24_E[12, 10]_B18_D[12, 14]	13.84152912
 L24_E[12, 10]_B12_D[14, 16]	13.84234178
 L24_E[18, 16]_B12_D[8, 10]	13.85278676
 L24_E[20, 18]_B8_D[12, 14]	13.86412837
 L24_E[12, 10]_B16_D[16, 18]	13.87255264
 L24_E[12, 10]_B16_D[8, 10]	13.93744659
 L24_E[10, 8]_B8_D[10, 12]	13.96210396
 L24_E[16, 14]_B12_D[16, 18]	13.96738719
 L24_E[10, 8]_B10_D[14, 16]	13.97912605
 L24_E[12, 10]_B8_D[10, 12]	14.01660612
 L24_E[14, 12]_B10_D[14, 16]	14.02922715
 L24_E[10, 8]_B12_D[10, 12]	14.04766726
 L24_E[12, 10]_B12_D[20, 22]	14.066748
 L24_E[12, 10]_B8_D[14, 16]	14.06781638
 L24_E[20, 18]_B12_D[8, 10]	14.08943156
 L24_E[14, 12]_B12_D[8, 10]	14.0936597
 L24_E[20, 18]_B8_D[8, 10]	14.18004001
 L24_E[12, 10]_B8_D[12, 14]	14.21057316
 L24_E[10, 8]_B10_D[16, 18]	14.26066824
 L24_E[12, 10]_B14_D[16, 18]	14.30954578
 L24_E[16, 14]_B8_D[18, 20]	14.31812593
 L24_E[16, 14]_B8_D[8, 10]	14.32220705
 L24_E[14, 12]_B8_D[10, 12]	14.37428971
 L24_E[12, 10]_B14_D[8, 10]	14.40916672
 L24_E[10, 8]_B12_D[20, 22]	14.41986412
 L24_E[14, 12]_B8_D[14, 16]	14.42364743
 L24_E[12, 10]_B14_D[14, 16]	14.51027182
 L24_E[14, 12]_B10_D[18, 20]	14.52233413
 L24_E[14, 12]_B14_D[12, 14]	14.54974282
 L24_E[18, 16]_B18_D[8, 10]	14.61622003
 L24_E[22, 20]_B12_D[8, 10]	14.62560012
 L24_E[22, 20]_B16_D[8, 10]	14.6842582
 L24_E[12, 10]_B8_D[8, 10]	14.69994849
 L24_E[12, 10]_B18_D[16, 18]	14.72891924
 L24_E[14, 12]_B12_D[10, 12]	14.76108782
 L24_E[10, 8]_B8_D[20, 22]	14.91739082
 L24_E[18, 16]_B8_D[10, 12]	14.92869174
 L24_E[10, 8]_B14_D[10, 12]	14.98058194
 L24_E[10, 8]_B18_D[12, 14]	15.49415174
 L24_E[18, 16]_B8_D[16, 18]	15.59678301
 L24_E[18, 16]_B10_D[8, 10]	16.26595131
 L24_E[12, 10]_B16_D[12, 14]	16.79394661
 L24_E[10, 8]_B12_D[8, 10]	16.80334629
 L24_E[10, 8]_B14_D[8, 10]	17.22927179
 L24_E[10, 8]_B8_D[16, 18]	17.43978841
 L24_E[16, 14]_B14_D[8, 10]	18.35258969
 L24_E[10, 8]_B18_D[16, 18]	18.42101433
 L24_E[12, 10]_B10_D[10, 12]	18.56789777
 L24_E[12, 10]_B16_D[18, 20]	18.58015577
 L24_E[12, 10]_B10_D[8, 10]	18.59265319
 L24_E[18, 16]_B8_D[12, 14]	19.10582537
 L24_E[10, 8]_B10_D[18, 20]	19.17482698
 L24_E[10, 8]_B8_D[8, 10]	19.89044466
 L24_E[10, 8]_B10_D[8, 10]	19.97303186
 L24_E[10, 8]_B16_D[14, 16]	20.18635533
 L24_E[10, 8]_B16_D[8, 10]	20.29022377
 L24_E[10, 8]_B18_D[10, 12]	20.89389116
 L24_E[10, 8]_B10_D[20, 22]	29.18064234
 L24_E[14, 12]_B16_D[14, 16]	52.72834657
'''